# Migration remove brasilian-pt accounts

In [ ]:
# required imports to access api_db, misc, misc.CONFIG, ...
import sys
sys.path = ['.', '..', '../..'] + sys.path
from collection import *

<hr>
<h1 align="center">driver code</h1>

Run all necessary instructions to properly setup the database (idempotent operations)

In [ ]:
foreign_locations = ["rio de janeiro", "salvador", "itabuna", "bahia", "recife", "copa", "fortaleza", "manaus", "brazil", "sao paulo", "são paulo", "maracanã", "maracana", "belo horizonte", "🇧🇷", "mato grosso", "mato g.", "curitiba", "alagoinhas", "rj", "brasilia", "brasília", "porto alegre", "brazuca" "-br", "🇪🇸", "spain", "madrid", "🇺🇸", "espana", "españa", "italia", "italy", "rome", "greece", "athens", "aθήνα", "eλλάδα", "🇫🇷", "france", "paris"]
foreign_descriptions = ["brasil", "brazil", "bolsonaro", "lula", "🇧🇷", "brazuca", " br ", "🇪🇸", "🇺🇸", "🇫🇷"]

def is_foreign(user):
    loc = user["location"].lower() if "location" in user else ""
    desc = user["description"].lower() if "description" in user else ""
    if "portugal" in loc: return False
    
    if len(loc) and any(x in loc for x in foreign_locations): return True
    if len(desc) and any(x in desc for x in foreign_descriptions): return True
    return False

In [ ]:
find_params = find_exclude_invalid({"depth": {"$gt": 0}})
counter_1 = 0
for user in api_db.col_users.find(find_params, {"location": True, "description": True}):
    if is_foreign(user): # remove these users's tweets and then the users
        counter_1+=1
        api_db.col_tweets.remove({"user": user["_id"]})
        api_db.col_users.remove({"_id": user["_id"]})

In [ ]:
# remove accounts that are not tweeting in portuguese with more than 50 tweets and less than 10% of pt content
foreign_users = api_db.col_users.find({
    "count_parsed_tweets": {"$gte": 50},
    "most_common_language": {"$not": {"$in": ["pt", "und"]}},
    "$or": [
        {"tweeted_languages.pt": {"$exists": False}},
        {"tweeted_languages.pt": {"$lte": 5}}
    ]
}, {})
counter_2 = 0
for user in foreign_users:
    counter_2+=1
    api_db.col_tweets.remove({"user": user["_id"]})
    api_db.col_users.remove({"_id": user["_id"]})


In [ ]:
print("Removed %d foreign users with location and %d with most_common_language" % (counter_1, counter_2))

In [ ]:
print("DONE")